In [1]:
# # Load the Drive helper and mount
# from google.colab import drive

# # This will prompt for authorization.
# drive.mount('/content/drive')

# %cd '/content/drive/MyDrive/Colab Notebooks/C73_A2_1/data'
# !ls

# data_dir = '/content/drive/MyDrive/Colab Notebooks/C73_A2_1/data'
# result_dir = '/content/drive/MyDrive/Colab Notebooks/C73_A2_1/result'

In [2]:
data_dir = 'data'
result_dir = 'result'

# Import

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy

import warnings
warnings.filterwarnings("ignore")

# Read data

In [4]:
af = 'u'
pf = '7200-3600.__2__' #'10800-10800' #60 #30 #600 # 60  # seconds

Xtrain = np.load(f'{data_dir}/{af}4-3.tr.X.{pf}.npy', allow_pickle=True)
ytrain = np.load(f'{data_dir}/{af}4-3.tr.y.{pf}.npy', allow_pickle=True)

Xtest = np.load(f'{data_dir}/{af}4-3.t.X.{pf}.npy', allow_pickle=True)
ytest = np.load(f'{data_dir}/{af}4-3.t.y.{pf}.npy', allow_pickle=True)

# Xval = np.load(f'{data_dir}/{af}4-3.v.X.{pf}.npy', allow_pickle=True)
# yval = np.load(f'{data_dir}/{af}4-3.v.y.{pf}.npy', allow_pickle=True)

#? features names
fts_names = [line.strip() for line in open(f'{data_dir}/{af}4-3.fts_cols.{pf}.txt').readlines()]

In [5]:
# #? merge val with train
# Xtrain = np.concatenate((Xtrain, Xval))
# ytrain = np.concatenate((ytrain, yval))

# Features

### Use all features ?

In [6]:
# for i in range(0, len(fts_names)):
#     print(i, fts_names[i])

In [7]:
#? use all features
Xtrain = Xtrain[:,:]
Xtest = Xtest[:,:]

In [8]:
print(Xtrain.shape)
print(ytrain.shape)

(279418, 32)
(279418,)


In [9]:
np.where(ytrain == 1)

(array([  5948,   5949,   5950,   5951,   5952,   5953,   5954,   5955,
          5956,   5957,   5958,   5959,   5960,   5961,   8773,   8774,
          8775,   8776,   8777,   8778,   8779,   8780,   8781,   8782,
          8783,   8784,   8785,   8786, 134552, 134553, 134554, 134555,
        134556, 146613, 146614, 146615, 146616, 146617, 146618, 157955,
        157956, 168797, 168798, 197323, 206629, 206630, 206631, 215831,
        215832, 215833, 215834, 215835, 215836, 215837, 215838, 224947,
        224948, 224949, 224950, 224951, 224952, 224953, 224954, 224955,
        234045, 234046, 234047, 258834, 274274], dtype=int64),)

### Features selection

In [10]:
# from sklearn.feature_selection import SelectKBest
# from sklearn.feature_selection import f_classif

# #? find best features
# ft_model = SelectKBest(score_func=f_classif, k='all')
# fit = ft_model.fit(Xtest, ytest)

# df_fts_scores = pd.DataFrame(fit.scores_)
# df_fts_names = pd.DataFrame(fts_names)

# df_fts = pd.concat([df_fts_names, df_fts_scores], axis=1)
# df_fts.columns = ['Ft', 'Score']
# print(df_fts.nlargest(15, 'Score'))

In [11]:
# from sklearn.feature_selection import SelectKBest
# from sklearn.feature_selection import f_classif

# #? find best features
# ft_model = SelectKBest(score_func=f_classif, k='all')
# fit = ft_model.fit(Xtrain, ytrain)

# df_fts_scores = pd.DataFrame(fit.scores_)
# df_fts_names = pd.DataFrame(fts_names)

# df_fts = pd.concat([df_fts_names, df_fts_scores], axis=1)
# df_fts.columns = ['Ft', 'Score']
# print(df_fts.nlargest(15, 'Score'))

#### Choose features

In [12]:
# #? select top features ?
selected_fts = [
    'BytesPerPkt_mean',
    'PktsPerSec_mean','BytesPerSec_mean',#'SrcBytesPerSec_mean',#'Dport_RU',
    'Sport_max','Sport_mean',
    'n_flows',
    'BytesPerPkt_max','BytesPerSec_max','SrcBytesPerSec_max',
    'P_tcp','P_udp','P_other',#'P_icmp',

    # 'S_CON', 'S_alltcp', 'S_INT', 'S_ECR', 'S_REQ', 'S_RED', 'S_other', 'S_ECO', #'S_URP', 'S_URH', 'S_RSP', 
    'S_CON', 'S_alltcp', 'S_INT', 'S_RED', 'S_other', 'S_ECO', 
]

#? index of top features
ft_idx = [fts_names.index(ft) for ft in selected_fts]
print(ft_idx)

Xtrain = Xtrain[:,ft_idx]
Xtest = Xtest[:,ft_idx]
print(Xtrain.shape)

[3, 4, 5, 2, 1, 0, 6, 7, 8, 10, 9, 12, 13, 14, 15, 18, 31, 19]
(279418, 18)


# Prepare data for train

In [13]:
X_train = Xtrain[:]
y_train = ytrain[:]

X_test = Xtest[:]
y_test = ytest[:]

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(279418, 18) (279418,)
(18226, 18) (18226,)


In [14]:
#? prevent exceed memory
del Xtrain
del Xtest
# del Xval

# Now train

In [15]:
from sklearn.metrics import classification_report, confusion_matrix, PrecisionRecallDisplay, f1_score, roc_auc_score, roc_curve, auc

def evaluate(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    print(cm)
    print(classification_report(y_true, y_pred, digits=4))
    roc_auc = roc_auc_score(y_true, y_pred)
    print(roc_auc)

    # display = PrecisionRecallDisplay.from_predictions(y_true, y_pred)
    # _ = display.ax_.set_title("2-class Precision-Recall curve")

    return roc_auc

In [16]:
""" prepare data """
from sklearn.model_selection import train_test_split
from sklearn.utils import resample

nb_prediction = 1
np.random.seed(seed=123456)
tab_seed = np.random.randint(0, 1000000000, nb_prediction)

# X_train_new, y_train_new = resample(X_train, y_train, n_samples=X_train.shape[0]*10, random_state=tab_seed[0])

X_train_split, X_test_split, y_train_split, y_test_split = train_test_split(X_train, y_train, test_size=0.2, random_state=tab_seed[0])
X_train_new, y_train_new = resample(X_train_split, y_train_split, n_samples=X_train.shape[0]*10, random_state=tab_seed[0])

In [17]:
""" calculate class weighting """
from sklearn.utils.class_weight import compute_class_weight

weighting = compute_class_weight(class_weight='balanced', classes=[0,1], y=y_train_new)
weighting

array([5.00128156e-01, 1.95124302e+03])

In [18]:
from sklearn.linear_model import LogisticRegression

""" train """
# normal_weight = 0.024 #0.044
# c = 650 #550
# model = LogisticRegression(penalty='l2', C=c, random_state=tab_seed[0], multi_class='auto', solver='lbfgs', class_weight={0:normal_weight, 1:1-normal_weight}, max_iter=1000, verbose=0)
model = LogisticRegression(penalty='none', random_state=tab_seed[0], solver='lbfgs', class_weight='balanced', max_iter=2000, verbose=0)
# model = LogisticRegression(penalty='none', C=c, random_state=tab_seed[0], solver='lbfgs', class_weight='balanced', max_iter=9000, verbose=0)
model.fit(X_train_new, y_train_new)

LogisticRegression(class_weight='balanced', max_iter=2000, penalty='none',
                   random_state=545331265)

# Evaluate

In [19]:
""" predict and evaluate on train_split """
y_pred_train_new = model.predict(X_train_new)
_ = evaluate(y_train_new, y_pred_train_new)

[[2533435  260029]
 [      0     716]]
              precision    recall  f1-score   support

           0     1.0000    0.9069    0.9512   2793464
           1     0.0027    1.0000    0.0055       716

    accuracy                         0.9069   2794180
   macro avg     0.5014    0.9535    0.4783   2794180
weighted avg     0.9997    0.9069    0.9509   2794180

0.9534576067563427


In [20]:
""" predict and evaluate on test_split """
y_pred_test_split = model.predict(X_test_split)
_ = evaluate(y_test_split, y_pred_test_split)

[[50606  5263]
 [    0    15]]
              precision    recall  f1-score   support

           0     1.0000    0.9058    0.9506     55869
           1     0.0028    1.0000    0.0057        15

    accuracy                         0.9058     55884
   macro avg     0.5014    0.9529    0.4781     55884
weighted avg     0.9997    0.9058    0.9503     55884

0.9528987452791351


In [21]:
""" predict and evaluate on test """
y_pred_test = model.predict(X_test)
roc_auc = evaluate(y_test, y_pred_test)

[[17219   947]
 [    0    60]]
              precision    recall  f1-score   support

           0     1.0000    0.9479    0.9732     18166
           1     0.0596    1.0000    0.1125        60

    accuracy                         0.9480     18226
   macro avg     0.5298    0.9739    0.5429     18226
weighted avg     0.9969    0.9480    0.9704     18226

0.9739348232962677


# Save predicted output

In [23]:
import pickle

ra = '{:.2f}'.format(round(roc_auc * 100, 2))

#? save model
pickle.dump(model, open(f'{result_dir}/{af}4-5.{pf}.model_lr.__{ra}__.pkl','wb'))

#? save predicted output for test set
np.save(f'{result_dir}/{af}4-5.{pf}.output_lr.__{ra}__.t.y_pred.npy', y_pred_test)

#? save data
np.save(f'{result_dir}/{af}4-5.{pf}.data_lr.__{ra}__.tr.X.npy', X_train)
np.save(f'{result_dir}/{af}4-5.{pf}.data_lr.__{ra}__.tr.y.npy', y_train)
np.save(f'{result_dir}/{af}4-5.{pf}.data_lr.__{ra}__.t.X.npy', X_test)
np.save(f'{result_dir}/{af}4-5.{pf}.data_lr.__{ra}__.t.y.npy', y_test)

# #? save config
# open(f'{result_dir}/{af}4-5.{pf}.config_lr.__{ra}__.txt', 'w').write(f'c={c}')

#? save features
open(f'{result_dir}/{af}4-5.{pf}.fts_lr.__{ra}__.txt', 'w').write('\n'.join(selected_fts))

190

In [24]:
#? cleanup
del X_train
del y_train
del X_train_new
del y_train_new
del X_test
del y_test
del y_pred_test
del model